In [1]:
import os

import numpy as np
import torch
import pandas as pd
import pysrt
from pandas import DataFrame
from torch.utils.data import Dataset, DataLoader

In [2]:
df = pd.read_csv("../data/row/table_pazzle.csv", dtype=str, index_col=0)
df.shape

(3561, 4)

In [3]:
df.sample(3)

,name,file_name,num_file,level
0,Armageddon,Armageddon.Time.2022.1080p.BluRay.REMUX.AVC.DT...,False,middle
0,Den of Thieves,Den.of.Thieves.2018.Concorde.Cut.GER.BD.Remux....,False,hard
0,Coffee and Cigarettes,Coffee.And.Cigarettes.2003.1080p.BluRay.x264-Y...,False,hard


In [4]:
df.dropna(inplace=True)
df.shape

(3504, 4)

In [5]:
df["level"].value_counts()

level
middle    1831
easy      1105
hard       568
Name: count, dtype: int64

In [6]:
name = "owl"
os.path.exists(f"../data/row/sub/{name}")

False

In [7]:
# убираем данные о нечитаемых субтитрах
error_file = []
for name in df["file_name"]:
    if os.path.exists(f"../data/row/sub/{name}"):
        try:
            os.path.exists(f"../data/row/sub/{name}")
            read_src = pysrt.open(f"../data/row/sub/{name}", encoding="iso-8859-1")
            if len(read_src) == 0:
                error_file.append(name)
        except OSError:
            error_file.append(name)
    else:
        error_file.append(name)
len(error_file)

43

In [16]:
len(pysrt.open(f"../data/row/sub/YsMn.Br.srt", encoding="iso-8859-1"))

1397

NameError: name 'SubRipFile' is not defined

In [11]:
"owl.csv" in error_file

False

In [8]:
error_file = []
for name in df["file_name"]:
    pysrt.open(f"../data/row/sub/{name}", encoding="iso-8859-1")

FileNotFoundError: [Errno 2] No such file or directory: '../data/row/sub/d0pe-potcdmcrpk1.srt,d0pe-potcdmcrpk2.srt'

In [ ]:
df = df[df.file_name.isin(error_file) == False]
df.shape

(3461, 4)

In [ ]:
df["level"].value_counts(normalize=True)

level
middle    0.523548
easy      0.314360
hard      0.162092
Name: proportion, dtype: float64

In [ ]:
df.to_csv("../data/interim/final_table.csv")

## Делаем эмбединги

In [2]:
df = pd.read_csv("../data/interim/culling_table_pazzle.csv")
df.sample(3)

,name,file_name,num_file,level
1903,Mass,Mass.Effect.Paragon.Lost.2012.BluRay.1080p.DTS...,False,middle
2570,Superbad,Superbad.2007.UNRATED.1080p.BluRay.x265-RARBG....,False,middle
2455,The Mosquito Coast,The Mosquito Coast (1986).srt,False,middle


In [4]:
import re

from transformers import AutoTokenizer, AutoModelForMaskedLM

In [5]:
df = pd.read_csv("../data/interim/culling_table_pazzle.csv")
it_names = iter(df["file_name"])

In [58]:

for scr_name in it_names:
    break

In [70]:
one = f'[^A-Za-z0-9.,?!:\' ]+'
two = f"<i>|</i>|\n|Â¶|â\x99ª"

In [71]:
qw = pysrt.open(f"../data/raw/sub/{scr_name}", encoding="iso-8859-1")
text = [
    re.sub(one, "", i.text).strip() for i in qw
]
text

['Doc!',
 'Doc, Doc, Doc!',
 'Doc! What?',
 'Doc!',
 'Okay, relax, Doc.',
 "It's me. It's me!It's Marty.",
 "No, it can't be.",
 'I just sent you backto the future.',
 'No, I know.You did send me back...',
 "to the future,but I'm back.",
 "I'm back from the future.",
 'Great Scott!',
 'iSay kids, what time is it?i',
 'iHowdy Doody time!i',
 "i It's Howdy i",
 'Great Scott! i Doody Time i',
 "i It's Howdy Doody Time i",
 'i Bob Smith and Howdy too i',
 'i Say Howdy Do to you i',
 'Howdy Doody time?',
 'Date: Sunday,',
 'November 13, 1955.7:01 a.m.',
 "Last night's time travel experimentwas apparently a complete success.",
 'Lightning struck the clocktower at precisely 10:04 p.m.,',
 'sending the necessary 1.21gigawatts into the time vehicle,',
 'which vanished ina brilliant flash of light...',
 'leaving a pair of fire trails behind.',
 'I, therefore, assume...',
 'that Marty and the time vehiclewere transported forward...',
 'through time into the year 1985.',
 'After that,',
 'uh, afte

In [36]:
class BuildDataset(Dataset):
    def __init__(self, list_text: list, tokenizer: object):
        super().__init__()
        self.list_text = list_text
        self.tokenizer = tokenizer

        self.list_text_size = []
        self.max_len = 512

    def set_len_text(self):
        for text in self.list_text:
            self.list_text_size = len(
                self.tokenizer.encode(text, add_special_tokens=True)
            )
            if max(self.list_text_size) < self.max_len:
                self.max_len = max(self.list_text_size)

    def __len__(self):
        return len(self.list_text)

    def __getitem__(self, index):
        token = self.tokenizer(
            self.list_text[index],
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            add_special_tokens=True,
        )

        return {
            "input_ids": token["input_ids"],
            "attention_mask": token["attention_mask"],
            "index": index,
        }

In [42]:
from tqdm.auto import tqdm

tqdm.pandas()


class Make_Features:
    def __init__(
        self, path_df: str, path_sub: str, path_save: str, batch_size: int, device: str
    ):
        self.path_df = path_df
        self.path_sub = path_sub
        self.path_save = path_save
        self.device = device
        self.df = pd.read_csv(path_df)

        self.list_emb = []
        self.bert_base_list = [
            "bert-base-uncased",
            "bert-large-uncased",
            "albert-base-v2",
            "albert-xxlarge-v1",
        ]

        self.reg_for_remove = f"<i>|</i>|\n|Â¶|â\x99"
        self.reg_for_space = f"'"
        self.batch_size = batch_size

    def __export_tokenazer_model(self, name_model):
        self.tokenizer = AutoTokenizer.from_pretrained(name_model)
        self.model = AutoModelForMaskedLM.from_pretrained(name_model).to(self.device)

    def __remove_spec_symbols(self, text):
        text = re.sub(self.reg_for_remove, " ", text.text)
        text = re.sub(self.reg_for_space, "", text)
        return text.strip()

    def __make_text_list(self):
        self.text_list = []
        for scr_name in tqdm(self.df["file_name"]):
            text = pysrt.open(f"{self.path_sub}/{scr_name}", encoding="iso-8859-1")
            part_text_list = []
            for part_text in text:
                part_text_list.append(self.__remove_spec_symbols(part_text))
            self.text_list.append(" ".join(part_text_list))

    def __get_embedded(self, train_load):
        for batch in tqdm(train_load):
            with torch.no_grad():
                token = torch.squeeze(batch["input_ids"], dim=1).to(self.device)
                mask = torch.squeeze(batch["attention_mask"], dim=1).to(self.device)
                batch_embeddings = self.model(token, attention_mask=mask)
                self.list_emb.append(batch_embeddings[0][:, 0, :].cpu().numpy())

    def run(self):
        print("preparing the texts")
        self.__make_text_list()

        for name_model in self.bert_base_list:
            print(f"get embeddings with {name_model}")
            self.__export_tokenazer_model(name_model)
            data = BuildDataset(self.text_list, self.tokenizer)
            train_load = DataLoader(data, batch_size=self.batch_size, num_workers=0)
            self.__get_embedded(train_load)

            embed_np = np.concatenate(self.list_emb)
            self.list_emb = []
            df_emb_text = pd.DataFrame(embed_np)
            df_emb_text.to_csv(f"{self.path_save}/{name_model}.csv", index=False)

In [43]:
test_class = Make_Features(
    "../data/interim/culling_table_pazzle.csv",
    "../data/raw/sub",
    "../data/interim",
    100,
    "cuda:0",
)

In [44]:
test_class.run()

preparing the texts


  0%|          | 0/3449 [00:00<?, ?it/s]

get embeddings with bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/35 [00:00<?, ?it/s]

...

  0%|          | 0/35 [00:00<?, ?it/s]

get embeddings with albert-base-v2


  0%|          | 0/35 [00:00<?, ?it/s]

get embeddings with albert-xxlarge-v1


KeyboardInterrupt: 

In [ ]:
test_class.text_list[0]

'Viv, I know there are only three shopping days left, but they called, And they want to see your house in an hour. (rock music playing) Uh-huh. Mom! Make Buzz turn off the music. (pop music playing) No problem. Mom! - "Mom." - Hang on, Viv. Let your sister rehearse. Like thatll help. And clean up the kitchen.  But... Now. Okay, Viv. Ill see you in a bit. I thought I told you to clean up the kitchen. I did. Can I go to the movies tonight with some friends? No. No... No, no. Im showing a house and I need you to baby-sit Kevin. Well, why cant Megan do it? Because she has to go to rehearsal at the teen center. And you dont  have  to go to the movies. I do if I want a life. Youre baby-sitting Kevin. Hey, where is he, anyway? Kevin. What is it? What is it?  BOY:  A car! Whoa!  MAN:  Wait a second. That was... Honey, that was mine. BUZZ (on tv):  Awesome. Its got an antenna and everything. DAD:  Merry Christmas, Mommy.  BOY:  Merry Christmas. MOM (on tv):  Merry Christmas. DAD:  What did I ge